## Feature Extraction

In [1]:
from chevy.features import KingSafety, PawnStructure, BoardFeatures

class FeatureExtractor:
    def __init__(self, board):
        self.board = board
        self.king_features = [
            "king_mobility", "king_centrality", "king_attackers_looking_at_ring_1",
            "king_defenders_at_ring_1", "checked", "castling_rights"
        ]
        self.pawn_features = [
            "passed_pawns", "isolated_pawns", "blocked_pawns", "central_pawns"
        ]
        self.board_features = [
            "bishop_pair", "fianchetto_queen", "fianchetto_king", "queens_mobility",
            "open_files_rooks_count", "connected_rooks", "connectivity"
        ]

    def extract_features(self, color):
        """Extract all positional features for the current board state."""
        king_features = KingSafety(self.board, color=color)
        pawn_features = PawnStructure(self.board, color=color)
        board_features = BoardFeatures(self.board, color=color)

        feature_values = []
        for feature in self.king_features:
            value = getattr(king_features, feature)
            feature_values.append((feature, value))

        for feature in self.pawn_features:
            value = getattr(pawn_features, feature)
            feature_values.append((feature, value))

        for feature in self.board_features:
            value = getattr(board_features, feature)
            feature_values.append((feature, value))

        numerical_features = {}
        for feature, value in feature_values:
            if isinstance(value, int):
                numerical_features[feature] = value
            elif isinstance(value, list):
                array_sum = sum(value)
                numerical_features[feature] = array_sum

        return numerical_features


In [2]:
import chess
import chess.engine
import pandas as pd
from tqdm import tqdm

engine = chess.engine.SimpleEngine.popen_uci("stockfish.exe") 

def evaluate_position(board):
    info = engine.analyse(board, chess.engine.Limit(time=0.05))
    return info["score"].relative.score(mate_score=10000) if "score" in info else None


def process_pgn_file(file_path):
    file = open(file_path)
    games = []
    while True:
        game = chess.pgn.read_game(file)
        if game is None:
            break
        games.append(game)
    print(f"Fetched {len(games)} games from {file_path}.")
    
    with tqdm(total=len(games), desc="Processing PGN File") as pbar:
        positions = []
        for game in games:
            board = game.board()
            feature_extractor = FeatureExtractor(board)
            for move in game.mainline_moves():
                board.push(move)
                features = feature_extractor.extract_features(chess.WHITE)
                evaluation = evaluate_position(board)
                if evaluation is not None:
                    features["evaluation_score"] = evaluation
                    positions.append(features)
            pbar.update(1)
    return positions

all_positions = []
for file_path in ["data/1.pgn", "data/2.pgn", "data/3.pgn"]:
    all_positions.extend(process_pgn_file(file_path))

df = pd.DataFrame(all_positions)
df.to_csv("data/chess_positions.csv", index=False)

engine.quit()

Fetched 6640 games from data/1.pgn.


Processing PGN File: 100%|██████████| 6640/6640 [8:12:43<00:00,  4.45s/it]   


Fetched 6430 games from data/2.pgn.


Processing PGN File:  90%|████████▉ | 5781/6430 [7:16:49<38:19,  3.54s/it]   

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_csv('data/chess_positions.csv')

X = df.drop(columns=['evaluation_score'])
y = df['evaluation_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

feature_weights = pd.Series(model.coef_, index=X.columns)
print("Feature Weights:")
print(feature_weights)

normalized_weights = np.abs(feature_weights) / np.sum(np.abs(feature_weights))
print("Normalized Feature Weights:")
print(normalized_weights)